In [1]:
#adapted from https://github.com/aweitz/EveryNoise/blob/master/scrapGenres.ipynb
from bs4 import BeautifulSoup
import requests, sys
import pandas as pd

In [2]:
r = requests.get("http://everynoise.com/everyschool.cgi?scope=US")

In [34]:
# Link here and read into Python, or just an html link as markdown.
# ?root=The%20University%20of%20Texas%20at%20Austin&
import re
norm = re.compile('[+»&\s-]')
soup = BeautifulSoup(r.text,"html.parser")
allSchools = soup.find_all("tr", "datarow")
## Cycle through genres and go to that genres page
schools = schools[:-1]
# popular_genres = []
# playlists = []
# schCnt = 0
for schoolRow in allSchools[629:]:
    print("Pulling school #" + str(schCnt), end = '\t')
    sys.stdout.flush()
    school = schoolRow.text.split('USA')[1].strip()
    print(school)
    schools.append(school)
    schoolPage = "http://everynoise.com/everyschool.cgi?root=" + school.replace(' ','%20') + "&scope=US"
    ## Pull artists from genre page
    r2 = requests.get(schoolPage)
    soup2 = BeautifulSoup(r2.text,"html.parser")
    schools_genre_div = soup2.find_all("div","note")
    inner = soup2.find_all("div","note")
    for d in inner:
        genres = d.text.split('\n')[1:-1]
        genres = [re.sub(norm,'',g) for g in genres]
    popular_genres.append(genres)
    spotify = soup2.find("iframe")
    playlists.append(spotify['src'])
    schCnt = schCnt+1

Pulling school #629	North Carolina Central University
Pulling school #630	School of the Art Institute of Chicago
Pulling school #631	Xavier University of Louisiana
Pulling school #632	Morgan State University
Pulling school #633	Benedictine College
Pulling school #634	Universidad Politecnica de Puerto Rico
Pulling school #635	Yeshiva University
Pulling school #636	Hawaii Pacific University
Pulling school #637	Inter American University of Puerto Rico-Bayamon
Pulling school #638	Taylor University
Pulling school #639	Hampton University
Pulling school #640	Regent University
Pulling school #641	Andrews University
Pulling school #642	Berklee College of Music
Pulling school #643	Moody Bible Institute
Pulling school #644	University of Hawaii-West Oahu
Pulling school #645	Vanguard University of Southern California
Pulling school #646	John Brown University
Pulling school #647	Spelman College
Pulling school #648	Evangel University
Pulling school #649	Rhode Island School of Design
Pulling school #6

In [35]:
print(len(schools),len(popular_genres),len(playlists))
df = pd.DataFrame({'SCHOOL':schools, 'GENRES':popular_genres, 'PLAYLIST_LINK':playlists})
df.to_csv('school_genres.csv')
df

675 675 675


,SCHOOL,GENRES,PLAYLIST_LINK
0,Texas A & M University-College Station,"[reddirt, outlawcountry, texascountry, anthemw...",https://embed.spotify.com/?uri=spotify:playlis...
1,The University of Texas at Austin,"[modernrock, indiepop, pop, indiepoptimism, al...",https://embed.spotify.com/?uri=spotify:playlis...
2,Ohio State University-Main Campus,"[pop, indiepoptimism, modernrock, indiepop, po...",https://embed.spotify.com/?uri=spotify:playlis...
3,University of Central Florida,"[pop, poprap, rap, hiphop, electrohouse, elect...",https://embed.spotify.com/?uri=spotify:playlis...
4,Arizona State University-Tempe,"[edm, electrohouse, pop, electronictrap, brost...",https://embed.spotify.com/?uri=spotify:playlis...
5,University of Florida,"[pop, modernrock, indiepoptimism, indiepop, po...",https://embed.spotify.com/?uri=spotify:playlis...
6,University of Wisconsin-Madison,"[indiepoptimism, pop, electropop, modernrock, ...",https://embed.spotify.com/?uri=spotify:playlis...
7,University of California-Los Angeles,"[alternativerb, pop, indierb, edm, electrohous...",https://embed.spotify.com/?uri=spotify:playlis...
8,University of Michigan-Ann Arbor,"[indiepoptimism, pop, modernrock, electropop, ...",https://embed.spotify.com/?uri=spotify:playlis...
9,University of Minnesota-Twin Cities,"[modernrock, pop, indiepoptimism, indiepop, el...",https://embed.spotify.com/?uri=spotify:playlis...


In [37]:
r = requests.get("http://everynoise.com/engenremap.html")

In [57]:
import re

soup = BeautifulSoup(r.text,"html.parser")
allGenreDivs = soup.find_all("div", "genre scanme")

## Cycle through genres and go to that genres page
genreListArtist = []
genreListOpp = []
genreListSim = []
genreList = []
playlists = []
simWeights = []
oppWeights = []
artistWeights = []
genreCnt = 0
for genreDiv in allGenreDivs:
    print("Pulling genre #" + str(genreCnt))
    sys.stdout.flush()
    genre = (genreDiv.text)
    genre = re.sub("[:'+»&\s-]", '', genre)    
    genreList.append(genre)
    genrePage = "http://everynoise.com/engenremap-" + genre + ".html"

    ## Pull artists from genre page
    r2 = requests.get(genrePage)
    soup2 = BeautifulSoup(r2.text,"html.parser")
    spotify_link = soup2.find_all("a", text = 'playlist')
    playlists.append(spotify_link[0]['href'])
    allArtistDivs = set(soup2.find_all("div", "genre scanme"))
    allGenresRelated = set(soup2.find_all("div", "genre"))
    allGenresRelated = allGenresRelated - allArtistDivs
    artistList = []
    art_w = []
    
    for artist in allArtistDivs:
        weight = artist['style'].split()[-1].replace('%', '')
        artistName = artist.text.strip().replace("»", "").replace(' ', '')
        if not(artistName.isspace()):   
            art_w.append(weight)
            artistList.append(artistName) 
    artistWeights.append(art_w)
    genreListArtist.append(artistList)
    
    simGenres = []
    oppGenres = []
    weights_sim = []
    weights_opp = []
    
    for other_genre in allGenresRelated:
        weight = other_genre['style'].split()[-1].replace('%', '')
        if 'nearby' in other_genre['id']:
            weights_sim.append(weight)
            simGenres.append(other_genre.text.strip().replace("»", "").replace(' ', ''))
        elif 'mirror' in other_genre['id']:
            weights_opp.append(weight)
            oppGenres.append(other_genre.text.strip().replace("»", "").replace(' ', ''))
    genreListSim.append(simGenres)
    genreListOpp.append(oppGenres)
    simWeights.append(weights_sim)
    oppWeights.append(weights_opp)
    genreCnt = genreCnt+1

Pulling genre #0
Pulling genre #1
Pulling genre #2
Pulling genre #3
Pulling genre #4
Pulling genre #5
Pulling genre #6
Pulling genre #7
Pulling genre #8
Pulling genre #9
Pulling genre #10
Pulling genre #11
Pulling genre #12
Pulling genre #13
Pulling genre #14
Pulling genre #15
Pulling genre #16
Pulling genre #17
Pulling genre #18
Pulling genre #19
Pulling genre #20
Pulling genre #21
Pulling genre #22
Pulling genre #23
Pulling genre #24
Pulling genre #25
Pulling genre #26
Pulling genre #27
Pulling genre #28
Pulling genre #29
Pulling genre #30
Pulling genre #31
Pulling genre #32
Pulling genre #33
Pulling genre #34
Pulling genre #35
Pulling genre #36
Pulling genre #37
Pulling genre #38
Pulling genre #39
Pulling genre #40
Pulling genre #41
Pulling genre #42
Pulling genre #43
Pulling genre #44
Pulling genre #45
Pulling genre #46
Pulling genre #47
Pulling genre #48
Pulling genre #49
Pulling genre #50
Pulling genre #51
Pulling genre #52
Pulling genre #53
Pulling genre #54
Pulling genre #55
Pu

Pulling genre #437
Pulling genre #438
Pulling genre #439
Pulling genre #440
Pulling genre #441
Pulling genre #442
Pulling genre #443
Pulling genre #444
Pulling genre #445
Pulling genre #446
Pulling genre #447
Pulling genre #448
Pulling genre #449
Pulling genre #450
Pulling genre #451
Pulling genre #452
Pulling genre #453
Pulling genre #454
Pulling genre #455
Pulling genre #456
Pulling genre #457
Pulling genre #458
Pulling genre #459
Pulling genre #460
Pulling genre #461
Pulling genre #462
Pulling genre #463
Pulling genre #464
Pulling genre #465
Pulling genre #466
Pulling genre #467
Pulling genre #468
Pulling genre #469
Pulling genre #470
Pulling genre #471
Pulling genre #472
Pulling genre #473
Pulling genre #474
Pulling genre #475
Pulling genre #476
Pulling genre #477
Pulling genre #478
Pulling genre #479
Pulling genre #480
Pulling genre #481
Pulling genre #482
Pulling genre #483
Pulling genre #484
Pulling genre #485
Pulling genre #486
Pulling genre #487
Pulling genre #488
Pulling genr

Pulling genre #869
Pulling genre #870
Pulling genre #871
Pulling genre #872
Pulling genre #873
Pulling genre #874
Pulling genre #875
Pulling genre #876
Pulling genre #877
Pulling genre #878
Pulling genre #879
Pulling genre #880
Pulling genre #881
Pulling genre #882
Pulling genre #883
Pulling genre #884
Pulling genre #885
Pulling genre #886
Pulling genre #887
Pulling genre #888
Pulling genre #889
Pulling genre #890
Pulling genre #891
Pulling genre #892
Pulling genre #893
Pulling genre #894
Pulling genre #895
Pulling genre #896
Pulling genre #897
Pulling genre #898
Pulling genre #899
Pulling genre #900
Pulling genre #901
Pulling genre #902
Pulling genre #903
Pulling genre #904
Pulling genre #905
Pulling genre #906
Pulling genre #907
Pulling genre #908
Pulling genre #909
Pulling genre #910
Pulling genre #911
Pulling genre #912
Pulling genre #913
Pulling genre #914
Pulling genre #915
Pulling genre #916
Pulling genre #917
Pulling genre #918
Pulling genre #919
Pulling genre #920
Pulling genr

Pulling genre #1286
Pulling genre #1287
Pulling genre #1288
Pulling genre #1289
Pulling genre #1290
Pulling genre #1291
Pulling genre #1292
Pulling genre #1293
Pulling genre #1294
Pulling genre #1295
Pulling genre #1296
Pulling genre #1297
Pulling genre #1298
Pulling genre #1299
Pulling genre #1300
Pulling genre #1301
Pulling genre #1302
Pulling genre #1303
Pulling genre #1304
Pulling genre #1305
Pulling genre #1306
Pulling genre #1307
Pulling genre #1308
Pulling genre #1309
Pulling genre #1310
Pulling genre #1311
Pulling genre #1312
Pulling genre #1313
Pulling genre #1314
Pulling genre #1315
Pulling genre #1316
Pulling genre #1317
Pulling genre #1318
Pulling genre #1319
Pulling genre #1320
Pulling genre #1321
Pulling genre #1322
Pulling genre #1323
Pulling genre #1324
Pulling genre #1325
Pulling genre #1326
Pulling genre #1327
Pulling genre #1328
Pulling genre #1329
Pulling genre #1330
Pulling genre #1331
Pulling genre #1332
Pulling genre #1333
Pulling genre #1334
Pulling genre #1335


Pulling genre #1696
Pulling genre #1697
Pulling genre #1698
Pulling genre #1699
Pulling genre #1700
Pulling genre #1701
Pulling genre #1702
Pulling genre #1703
Pulling genre #1704
Pulling genre #1705
Pulling genre #1706
Pulling genre #1707
Pulling genre #1708
Pulling genre #1709
Pulling genre #1710
Pulling genre #1711
Pulling genre #1712
Pulling genre #1713
Pulling genre #1714
Pulling genre #1715
Pulling genre #1716
Pulling genre #1717
Pulling genre #1718
Pulling genre #1719
Pulling genre #1720
Pulling genre #1721
Pulling genre #1722
Pulling genre #1723
Pulling genre #1724
Pulling genre #1725
Pulling genre #1726
Pulling genre #1727
Pulling genre #1728
Pulling genre #1729
Pulling genre #1730
Pulling genre #1731
Pulling genre #1732
Pulling genre #1733
Pulling genre #1734
Pulling genre #1735
Pulling genre #1736
Pulling genre #1737
Pulling genre #1738
Pulling genre #1739
Pulling genre #1740
Pulling genre #1741
Pulling genre #1742
Pulling genre #1743
Pulling genre #1744
Pulling genre #1745


Pulling genre #2106
Pulling genre #2107
Pulling genre #2108
Pulling genre #2109
Pulling genre #2110
Pulling genre #2111
Pulling genre #2112
Pulling genre #2113
Pulling genre #2114
Pulling genre #2115
Pulling genre #2116
Pulling genre #2117
Pulling genre #2118
Pulling genre #2119
Pulling genre #2120
Pulling genre #2121
Pulling genre #2122
Pulling genre #2123
Pulling genre #2124
Pulling genre #2125
Pulling genre #2126
Pulling genre #2127
Pulling genre #2128
Pulling genre #2129
Pulling genre #2130
Pulling genre #2131
Pulling genre #2132
Pulling genre #2133
Pulling genre #2134
Pulling genre #2135
Pulling genre #2136
Pulling genre #2137
Pulling genre #2138
Pulling genre #2139
Pulling genre #2140
Pulling genre #2141
Pulling genre #2142
Pulling genre #2143
Pulling genre #2144
Pulling genre #2145
Pulling genre #2146
Pulling genre #2147
Pulling genre #2148
Pulling genre #2149
Pulling genre #2150
Pulling genre #2151
Pulling genre #2152
Pulling genre #2153
Pulling genre #2154
Pulling genre #2155


Pulling genre #2516
Pulling genre #2517
Pulling genre #2518
Pulling genre #2519
Pulling genre #2520
Pulling genre #2521
Pulling genre #2522
Pulling genre #2523
Pulling genre #2524
Pulling genre #2525
Pulling genre #2526
Pulling genre #2527
Pulling genre #2528
Pulling genre #2529
Pulling genre #2530
Pulling genre #2531
Pulling genre #2532
Pulling genre #2533
Pulling genre #2534
Pulling genre #2535
Pulling genre #2536
Pulling genre #2537
Pulling genre #2538
Pulling genre #2539
Pulling genre #2540
Pulling genre #2541
Pulling genre #2542
Pulling genre #2543
Pulling genre #2544
Pulling genre #2545
Pulling genre #2546
Pulling genre #2547
Pulling genre #2548
Pulling genre #2549
Pulling genre #2550
Pulling genre #2551
Pulling genre #2552
Pulling genre #2553
Pulling genre #2554
Pulling genre #2555
Pulling genre #2556
Pulling genre #2557
Pulling genre #2558
Pulling genre #2559
Pulling genre #2560
Pulling genre #2561
Pulling genre #2562
Pulling genre #2563
Pulling genre #2564
Pulling genre #2565


Pulling genre #2926
Pulling genre #2927
Pulling genre #2928
Pulling genre #2929
Pulling genre #2930
Pulling genre #2931
Pulling genre #2932
Pulling genre #2933
Pulling genre #2934
Pulling genre #2935
Pulling genre #2936
Pulling genre #2937
Pulling genre #2938
Pulling genre #2939
Pulling genre #2940
Pulling genre #2941
Pulling genre #2942
Pulling genre #2943
Pulling genre #2944
Pulling genre #2945
Pulling genre #2946
Pulling genre #2947
Pulling genre #2948
Pulling genre #2949
Pulling genre #2950
Pulling genre #2951
Pulling genre #2952
Pulling genre #2953
Pulling genre #2954
Pulling genre #2955
Pulling genre #2956
Pulling genre #2957
Pulling genre #2958
Pulling genre #2959
Pulling genre #2960
Pulling genre #2961
Pulling genre #2962
Pulling genre #2963
Pulling genre #2964
Pulling genre #2965
Pulling genre #2966
Pulling genre #2967
Pulling genre #2968
Pulling genre #2969
Pulling genre #2970
Pulling genre #2971
Pulling genre #2972
Pulling genre #2973
Pulling genre #2974
Pulling genre #2975


Pulling genre #3336
Pulling genre #3337
Pulling genre #3338
Pulling genre #3339
Pulling genre #3340
Pulling genre #3341
Pulling genre #3342
Pulling genre #3343
Pulling genre #3344
Pulling genre #3345
Pulling genre #3346
Pulling genre #3347
Pulling genre #3348
Pulling genre #3349
Pulling genre #3350
Pulling genre #3351
Pulling genre #3352
Pulling genre #3353
Pulling genre #3354
Pulling genre #3355
Pulling genre #3356
Pulling genre #3357
Pulling genre #3358
Pulling genre #3359
Pulling genre #3360
Pulling genre #3361
Pulling genre #3362
Pulling genre #3363
Pulling genre #3364
Pulling genre #3365
Pulling genre #3366
Pulling genre #3367
Pulling genre #3368
Pulling genre #3369
Pulling genre #3370
Pulling genre #3371
Pulling genre #3372
Pulling genre #3373
Pulling genre #3374
Pulling genre #3375
Pulling genre #3376
Pulling genre #3377
Pulling genre #3378
Pulling genre #3379
Pulling genre #3380
Pulling genre #3381
Pulling genre #3382
Pulling genre #3383
Pulling genre #3384
Pulling genre #3385


In [58]:
# genreListArtist = []
# genreListOpp = []
# genreListSim = []
# genreList = []
# playlists = []
df = pd.DataFrame({"GENRE":genreList, 
                   "SIM_GENRES":genreListSim,
                   "SIM_WEIGHTS":simWeights,
                   "OPP_GENRES":genreListOpp,
                   "OPP_WEIGHTS":oppWeights,
                   "REL_ARTISTS":genreListArtist, 
                   "ARTIST_WEIGHTS":artistWeights,
                   "SPOTIFY_URL":playlists})
df.to_csv('all_genres.csv')
df

,GENRE,SIM_GENRES,SIM_WEIGHTS,OPP_GENRES,OPP_WEIGHTS,REL_ARTISTS,ARTIST_WEIGHTS,SPOTIFY_URL
0,backgroundmusic,"[reiki, calminginstrumental, yoga, backgroundm...","[103, 120, 101, 240, 102, 125, 113, 100]","[mahraganat, divahouse, oldschoolukhiphop, cha...","[160, 100, 121, 110, 135, 136, 121, 111, 115, ...","[NoemiNucci, JuditheAbelsen, FabienneWillis, A...","[113, 117, 106, 131, 109, 115, 117, 110, 105, ...",https://open.spotify.com/user/thesoundsofspoti...
1,classicmoroccanpop,"[bluesrock, andalusianclassical, chaabimarocai...","[103, 107, 109, 100, 102, 103, 103, 101, 100, ...","[shiverpop, gravewave, tassieindie, estonianin...","[112, 112, 105, 102, 124, 100, 160, 110, 123, ...","[HajHusseinToulali, AbdelhadiBelkheyat, HadjHo...","[102, 103, 103, 108, 108, 102, 105, 103, 160, ...",https://open.spotify.com/user/thesoundsofspoti...
2,orthodoxchant,"[universitychoir, polishchoir, ukrainianchoir,...","[102, 105, 107, 104, 106, 240, 103, 101, 102, ...","[sambass, funkybreaks, hardbass, hardcorebreak...","[107, 110, 158, 100, 119, 111, 109, 146, 113, ...","[LegeArtis, PeterMichaelides, CapellaGregorian...","[114, 100, 160, 127, 115, 115, 108, 125, 107, ...",https://open.spotify.com/user/thesoundsofspoti...
3,tar,"[oud, persiantraditional, bansuri, tar, bandon...","[106, 120, 101, 240, 103, 113, 102, 102, 103, ...","[happyhardcore, aggrotech, nightcore, bouncyho...","[109, 107, 127, 114, 115, 120, 126, 121, 113, ...","[KiyarashSaket, YusefForutan, HamidMotebassem,...","[100, 100, 111, 100, 100, 100, 100, 101, 106, ...",https://open.spotify.com/user/thesoundsofspoti...
4,reggae,"[dancehall, brazilianreggae, skarevival, deepr...","[109, 105, 114, 101, 100, 127, 110, 103, 108, ...","[brazilianlo-firock, martialindustrial, powere...","[106, 107, 155, 121, 160, 108, 149, 147, 131, ...","[TheItals, MykalRose, JohnnieClark, Aswad, Lee...","[101, 102, 100, 105, 101, 105, 108, 100, 107, ...",https://open.spotify.com/user/thesoundsofspoti...
5,ukdub,"[reggae, ukdub, rootsreggae, ukreggae, dub, du...","[100, 240, 101, 101, 113, 119]","[neofolk, dungeonsynth, dronepsych, canadianex...","[102, 108, 118, 121, 136, 117, 118, 116, 118, ...","[Dubwise, TheBushChemists, WayneMcArthur, Brot...","[104, 117, 101, 160, 112, 145, 112, 113, 135, ...",https://open.spotify.com/user/thesoundsofspoti...
6,russianpostrock,"[post-rocklatinoamericano, frenchpost-rock, fr...","[105, 106, 100, 103, 102, 104, 102, 240, 103, ...","[cumbiaecuatoriana, novelty, frenchreggae, muz...","[103, 124, 146, 102, 110, 126, 146, 160, 153, ...","[ShowMeADinosaur, HumanFactor, Ð¡ÑÐµÐ½ÑÐÑÐ...","[111, 109, 100, 106, 125, 102, 100, 100, 100, ...",https://open.spotify.com/user/thesoundsofspoti...
7,vietnamesepop,"[peiindie, brazilianccm, singaporeanmandopop, ...","[100, 100, 100, 101, 100, 100, 100, 131, 101, ...","[doujin, jazztronica, drillandbass, japaneseja...","[116, 110, 117, 129, 129, 100, 101, 127, 103, ...","[LuuChiVy, HoangLan, GiaoLinh, HongVan, HuongT...","[118, 107, 115, 100, 113, 145, 104, 102, 116, ...",https://open.spotify.com/user/thesoundsofspoti...
8,bandinhas,"[tecnobrega, cumbiaboliviana, popreggaeton, or...","[100, 102, 100, 100, 100, 100, 114, 100, 240, ...","[experimentalpsych, polishexperimental, neocla...","[114, 136, 105, 126, 105, 103, 160, 113, 106, ...","[GrupoTradiÃ§Ã£o, BandaCosmoExpress, SuperBand...","[124, 106, 116, 123, 102, 117, 103, 105, 110, ...",https://open.spotify.com/user/thesoundsofspoti...
9,belarussianmetal,"[slovenianmetal, lithuanianmetal, slavicmetal,...","[101, 101, 100, 100, 100, 104, 100, 100, 100, ...","[barnalog, ukulele, chinderlieder, barnmusik, ...","[125, 100, 128, 139, 128, 160, 126, 151, 152, ...","[Kliodna, NightsideGlance, EvilUnleashed, Zmro...","[101, 104, 100, 101, 101, 102, 111, 108, 101, ...",https://open.spotify.com/user/thesoundsofspoti...
